In [1]:
import datetime
import sys
import pandas as pd
import numpy as np
import sklearn

# 读取数据
data  = pd.read_csv('dataset_temp/okxorderbook20240421_1_cleaned.csv')

def unique_second(data):
    # 每一秒只保留一个数据
    data['datetime'] = pd.to_datetime(data['server_time'], unit='ms')
    # 设置server_time为索引
    data.set_index('datetime', inplace=True)
    # 按秒对server_time进行分组，并选择每个组的第一个记录
    df_unique_second = data.groupby(data.index.floor('S')).first()
    # 如果需要，可以将结果转换回DataFrame，并重置索引
    df_unique_second = df_unique_second.reset_index()
    return df_unique_second


data = unique_second(data)

# 当前价格，就是卖一价
data['price'] = data['ask_1_price']



/var/folders/14/djzvbv312951ytxybb9bdcb00000gn/T/ipykernel_66650/2190754619.py:16: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  df_unique_second = data.groupby(data.index.floor('S')).first()


In [3]:
def get_price_rise_radio(data):
    # 从5秒 到 60 秒，步长为5秒
    for i in range(5,60,5 ):
        data['rise_radio_' + str(i)] = data['price'].pct_change(i)
    return data

# 计算价格上涨比例
data = get_price_rise_radio(data)
data

,datetime,server_time,ask_1_price,ask_2_price,ask_3_price,bid_3_price,bid_2_price,bid_1_price,ask_1_amount,ask_2_amount,...,rise_radio_10,rise_radio_15,rise_radio_20,rise_radio_25,rise_radio_30,rise_radio_35,rise_radio_40,rise_radio_45,rise_radio_50,rise_radio_55
0,2024-04-20 16:00:00,1713628800004,64806.1,64808.1,64810.0,64804.8,64805.8,64806.0,0.595363,0.319200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-20 16:00:01,1713628801004,64801.3,64802.0,64803.3,64800.3,64800.5,64800.8,1.125924,0.116300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-04-20 16:00:02,1713628802004,64800.0,64802.0,64803.3,64799.1,64799.2,64799.9,0.461067,0.076300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-04-20 16:00:03,1713628803004,64800.0,64802.0,64802.6,64799.1,64799.2,64799.9,1.431959,0.076300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-04-20 16:00:04,1713628804004,64800.0,64801.9,64802.0,64799.1,64799.2,64799.9,0.787670,0.045780,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2024-04-21 04:01:38,1713672098006,65180.3,65180.4,65180.6,65179.2,65179.9,65180.0,0.126510,0.001865,...,-0.000005,-0.000005,0.000127,0.000494,0.000150,0.000275,0.000057,-0.000107,-0.000118,0.000069
198,2024-04-21 04:01:39,1713672099006,65176.8,65177.4,65178.0,65176.1,65176.4,65176.7,0.813219,0.040000,...,-0.000186,-0.000058,-0.000049,0.000441,0.000074,0.000104,-0.000018,-0.000086,-0.000172,0.000015
199,2024-04-21 04:01:40,1713672100006,65176.8,65177.4,65178.0,65176.1,65176.4,65176.7,0.813219,0.040000,...,-0.000186,-0.000058,-0.000057,0.000235,0.000120,0.000043,-0.000080,-0.000086,-0.000172,0.000015
200,2024-04-21 04:01:41,1713672101106,65176.8,65177.4,65178.0,65176.1,65176.4,65176.7,0.725602,0.040000,...,-0.000186,-0.000058,-0.000058,0.000235,0.000267,-0.000014,-0.000080,0.000003,-0.000172,0.000015


In [10]:
# 计算量的比例

def get_weight_percentage(data_all):
    # 生成权重，总权重为100，分为三个，每个步长为10，
    weights = []
    for i in range(0,100,10):
        for j in range(0,100-i,10):
            weights.append([i,j,(100-i-j)])

    for weight in weights:
        weight_ask = (weight[0] * data_all['ask_1_amount'] + weight[1]  * data_all['ask_2_amount']  + weight[2]  * data_all['ask_3_amount'] )
        weight_bid = (weight[0] * data_all['bid_1_amount'] + weight[1] *  data_all['bid_2_amount'] + weight[2] *  data_all['bid_3_amount'])
        w_ask_over_bid = weight_ask/weight_bid
        w_ask_bid_gap = (weight_ask - weight_bid)/(weight_ask + weight_bid)
        data_all[f'w{weight[0]}_{weight[1]}_{weight[2]}_ask_over_bid'] = w_ask_over_bid
        data_all[f'w{weight[0]}_{weight[1]}_{weight[2]}_ask_bid_gap'] = w_ask_bid_gap

    return data_all

get_weight_percentage(data)

,datetime,server_time,ask_1_price,ask_2_price,ask_3_price,bid_3_price,bid_2_price,bid_1_price,ask_1_amount,ask_2_amount,...,w60_0_40_ask_over_bid,w60_10_30_ask_over_bid,w60_20_20_ask_over_bid,w60_30_10_ask_over_bid,w70_0_30_ask_over_bid,w70_10_20_ask_over_bid,w70_20_10_ask_over_bid,w80_0_20_ask_over_bid,w80_10_10_ask_over_bid,w90_0_10_ask_over_bid
0,2024-04-20 16:00:00,1713628800004,64806.1,64808.1,64810.0,64804.8,64805.8,64806.0,0.595363,0.319200,...,1.024760,1.115069,1.209815,1.309333,1.032017,1.112167,1.195796,1.037815,1.109860,1.042552
1,2024-04-20 16:00:01,1713628801004,64801.3,64802.0,64803.3,64800.3,64800.5,64800.8,1.125924,0.116300,...,3.491739,3.874377,4.341204,4.923437,4.174360,4.642204,5.217457,4.907000,5.472322,5.695366
2,2024-04-20 16:00:02,1713628802004,64800.0,64802.0,64803.3,64799.1,64799.2,64799.9,0.461067,0.076300,...,0.827614,0.794373,0.764213,0.736726,0.826430,0.797152,0.770282,0.825505,0.799346,0.824763
3,2024-04-20 16:00:03,1713628803004,64800.0,64802.0,64802.6,64799.1,64799.2,64799.9,1.431959,0.076300,...,41.910930,22.842042,15.526817,11.658303,62.812108,29.875387,19.388892,104.529734,39.599293,229.008853
4,2024-04-20 16:00:04,1713628804004,64800.0,64801.9,64802.0,64799.1,64799.2,64799.9,0.787670,0.045780,...,1.771386,1.654018,1.550053,1.457320,1.784074,1.679083,1.584815,1.794092,1.699128,1.802203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197,2024-04-21 04:01:38,1713672098006,65180.3,65180.4,65180.6,65179.2,65179.9,65180.0,0.126510,0.001865,...,12.450111,8.901415,5.914486,3.365712,8.702122,5.862473,3.422236,5.814870,3.474291,3.522389
198,2024-04-21 04:01:39,1713672099006,65176.8,65177.4,65178.0,65176.1,65176.4,65176.7,0.813219,0.040000,...,5.039655,4.139455,3.504567,3.032764,4.968546,4.183544,3.606642,4.914551,4.218475,4.872155
199,2024-04-21 04:01:40,1713672100006,65176.8,65177.4,65178.0,65176.1,65176.4,65176.7,0.813219,0.040000,...,5.310035,4.742576,4.278823,3.892728,6.204539,5.533527,4.987447,7.141236,6.357878,8.123185
200,2024-04-21 04:01:41,1713672101106,65176.8,65177.4,65178.0,65176.1,65176.4,65176.7,0.725602,0.040000,...,1.997766,1.814913,1.660522,1.528427,1.957080,1.800968,1.666214,1.926399,1.790229,1.902437
